# IMDb 데이터셋을 활용한 BERT Fine-tuning 실습

- 파인 튜닝된 모델 활용해 리뷰의 감정을 예측 가능. BERT 모델을 특정 작업에 맞고 조정, 그 성능을 확인 가능하다.

## 필요 라이브러리 설치 및 임포트

1. 필요 라이브러리 설치
```
pip install transformers datasets torch
pip isntall accelerate -U
```

2. 라이브러리 임포트

In [1]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch

c:\Users\By the book\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## IMDb dataset load

- datasets 라이브러리를 사용, IMDb 영화 리뷰 데이터셋 로드.

In [2]:
# IMDb 데이터셋 로드
dataset = load_dataset("imdb")

# 훈련 및 테스트 데이터셋 분리
train_dataset = dataset['train'].shuffle(seed=42).select(range(1000))  # 1000개 샘플로 축소
test_dataset = dataset['test'].shuffle(seed=42).select(range(500))  # 500개 샘플로 축소

## 데이터 전처리 및 토크나이저 적용

- BERT 모델에 맞게 데이터를 토큰화.

In [3]:
# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 데이터셋 토크나이징 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# 데이터셋 토크나이징 적용
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# 모델 입력으로 사용하기 위해 데이터셋 포맷 설정
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

## BERT 모델 로드 및 Fine-tuning

- 사전 학습된 BERT 모델을 로드, IMDb 데이터셋을 사용해 Fine-tuning 진행.

In [5]:
# BERT 모델 로드
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_steps=10_000,
    save_total_limit=2,
)

# 트레이너 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# 모델 훈련
trainer.train()
trainer.evaluate()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                  
 33%|███▎      | 125/375 [27:20<47:40, 11.44s/it]

{'eval_loss': 0.40655940771102905, 'eval_runtime': 182.1555, 'eval_samples_per_second': 2.745, 'eval_steps_per_second': 0.346, 'epoch': 1.0}


                                                   
 67%|██████▋   | 250/375 [52:07<21:27, 10.30s/it]

{'eval_loss': 0.3735009729862213, 'eval_runtime': 163.0, 'eval_samples_per_second': 3.067, 'eval_steps_per_second': 0.387, 'epoch': 2.0}


                                                   
100%|██████████| 375/375 [1:16:03<00:00, 12.17s/it]


{'eval_loss': 0.4672318696975708, 'eval_runtime': 162.91, 'eval_samples_per_second': 3.069, 'eval_steps_per_second': 0.387, 'epoch': 3.0}
{'train_runtime': 4563.0182, 'train_samples_per_second': 0.657, 'train_steps_per_second': 0.082, 'train_loss': 0.3016983846028646, 'epoch': 3.0}


100%|██████████| 63/63 [02:39<00:00,  2.53s/it]


{'eval_loss': 0.4672318696975708,
 'eval_runtime': 161.759,
 'eval_samples_per_second': 3.091,
 'eval_steps_per_second': 0.389,
 'epoch': 3.0}

## 모델 평가 및 결과 확인

- Fine-tuning이 완료된 모델을 사용해 테스트 데이터셋으로 성능 평가.

In [6]:
import numpy as np
from sklearn.metrics import accuracy_score

# 평가 지표 함수 정의
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)  # 예측된 클래스
    labels = p.label_ids  # 실제 레이블
    acc = accuracy_score(labels, preds)  # 정확도 계산
    return {'accuracy': acc}

# 이미 훈련된 트레이너에 compute_metrics를 추가하여 평가
trainer.compute_metrics = compute_metrics

# 모델 평가 및 정확도 확인
eval_result = trainer.evaluate()
print(f"Accuracy: {eval_result['eval_accuracy']:.4f}")

100%|██████████| 63/63 [02:43<00:00,  2.59s/it]

Accuracy: 0.8980


## 새 리뷰 예측 해보기

- 새 리뷰를 토큰화해서 BERT 모델의 입력 형식으로 변환 후 예측값 반환.

In [7]:
def predict_review(review_text):
    # 입력 텍스트 토큰화
    inputs = tokenizer(review_text, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

    # 모델 예측 수행
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 소프트맥스 함수 적용 후 예측값 계산
    probs = torch.softmax(outputs.logits, dim=1)
    pred_label = torch.argmax(probs, dim=1).item()
    
    # 예측 결과 반환 (0: 부정, 1: 긍정)
    return "Positive" if pred_label == 1 else "Negative"

# 리뷰에 대한 예측
sample_review = "The movie was incredibly interesting and kept me on the edge of my seat!"
print("Prediction:", predict_review(sample_review))

Prediction: Positive
